# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



* José Alberto Mtanous Treviño
* Javier Muñoz Barrios
* José Francisco Muñoz Del Ángel
* César Alexis Nájera Mendoza
* Annette Cristina Narváez Andrade



In [1]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy

nltk.download('wordnet')
nlp = spacy.load('en_core_web_sm')



[nltk_data] Downloading package wordnet to /home/jmtanous/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [2]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
df_amazon = pd.read_csv("./data/amazon5.txt", sep='\t', names= ['review','tag'])
df_imdb = pd.read_csv("./data/imdb5.txt", sep=r'\s{3}', names= ['review','tag'], engine='python')
df_yelp = pd.read_csv("./data/yelp5.txt", sep='\t', names= ['review','tag'])

df = pd.concat([df_amazon, df_imdb, df_yelp])
# *********** Aquí termina la sección de agregar código *************


In [3]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   tag     3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [4]:
# Y veamos sus primeros registros:

df.head()

,review,tag
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [5]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
lemmatizer = WordNetLemmatizer()

lemmatized_tokens = []
lemmatized_reviews = []
for review in df['review']:
    review = re.sub(r'\.',r' ', review)
    review = re.sub(r'\s+',r' ', review)
    doc = nlp(review.lower())
    _lemmatized_tokens = [token.lemma_ for token in doc]
    _lemmatized_tokens = [token for token in _lemmatized_tokens if not re.match(r'\W', token)]
    lemmatized_tokens.append(_lemmatized_tokens)
    lemmatized_reviews.append(' '.join(_lemmatized_tokens))

df['lemmatized_tokens'] = lemmatized_tokens
df['lemmatized_reviews'] = lemmatized_reviews
df.head()
# *********** Aquí termina la sección de agregar código *************

,review,tag,lemmatized_tokens,lemmatized_reviews
0,So there is no way for me to plug it in here i...,0,"[so, there, be, no, way, for, I, to, plug, it,...",so there be no way for I to plug it in here in...
1,"Good case, Excellent value.",1,"[good, case, excellent, value]",good case excellent value
2,Great for the jawbone.,1,"[great, for, the, jawbone]",great for the jawbone
3,Tied to charger for conversations lasting more...,0,"[tie, to, charger, for, conversation, last, mo...",tie to charger for conversation last more than...
4,The mic is great.,1,"[the, mic, be, great]",the mic be great


In [6]:
x = df['lemmatized_reviews']

for m in x[3]:
    print(m)


tie to charger for conversation last more than 45 minute major problem
very little music or anything to speak of
stop by during the late may bank holiday off rick steve recommendation and love it


In [7]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

Xclean = df['lemmatized_tokens']

for x in Xclean[0:5]:
  print(x)


['so', 'there', 'be', 'no', 'way', 'for', 'I', 'to', 'plug', 'it', 'in', 'here', 'in', 'the', 'us', 'unless', 'I', 'go', 'by', 'a', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'for', 'the', 'jawbone']
['tie', 'to', 'charger', 'for', 'conversation', 'last', 'more', 'than', '45', 'minute', 'major', 'problem']
['the', 'mic', 'be', 'great']


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [8]:

# ************* Inicia la sección de agregar código:*****************************
from sklearn.model_selection import train_test_split
Y = df['tag']

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1) 
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

#print('X,y Train:', len(x_train), len(y_train))      # los "x_" son "list" y los "y_" son "Series"
#print('X,y Val:', len(x_val), len(y_val))
#print('X,y Test', len(x_test), len(y_test))


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [9]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

from collections import Counter

midiccionario = Counter()    

_x_train = x_train.to_list()

for k in range(len(_x_train)):
  midiccionario.update(_x_train[k])


# *********** Aquí termina la sección de agregar código *************

In [10]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********
print(len(midiccionario))

# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:
3526


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

Para evaluar correctamente el modelo, no debemos de mezclar datos de entrenamiento con los de validación o prueba. Si pretendemos usar el modelo en un caso real
no sabriamos de antemano el vocabulario completo.

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [11]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
train_x = x_train.to_list()
val_x = [token for token in x_val.to_list() if token in train_x]
test_x = [token for token in x_test.to_list() if token in train_x]

# *********** Aquí termina la sección de agregar código *************


In [12]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['his', 'co', 'star', 'do', 'not', 'fare', 'much', 'well', 'with', 'people', 'like', 'morgan', 'freeman', 'jonah', 'hill', 'and', 'ed', 'helm', 'just', 'waste']
['tonight', 'I', 'have', 'the', 'elk', 'filet', 'special', 'and', 'it', 'suck']
['I', 'pay', 'the', 'bill', 'but', 'do', 'not', 'tip', 'because', 'I', 'feel', 'the', 'server', 'do', 'a', 'terrible', 'job']
['how', 'can', 'you', 'call', 'yourself', 'a', 'steakhouse', 'if', 'you', 'can', 'not', 'properly', 'cook', 'a', 'steak', 'I', 'do', 'not', 'understand']
['however', 'the', 'keypad', 'be', 'so', 'tinny', 'that', 'I', 'sometimes', 'reach', 'the', 'wrong', 'button']


#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

| Modelo        | Pros                                                                 | Contras                                                                |
|---------------|----------------------------------------------------------------------|------------------------------------------------------------------------|
| **FastText**  | - Considera sub-palabras, lo que mejora la representación de palabras raras y errores ortográficos. <br> - Mejor rendimiento en idiomas morfológicamente ricos. <br> - Rápido y eficiente en términos de memoria. | - Mayor complejidad en comparación con Word2Vec. <br> - Entrenamiento más lento debido a la consideración de sub-palabras. |
| **Word2Vec**  | - Simplicidad y rapidez en el entrenamiento. <br> - Amplio uso y soporte en la comunidad. <br> - Dos modos de entrenamiento: Skip-gram y CBOW. | - No maneja bien palabras raras o errores ortográficos. <br> - No captura información de sub-palabras. |
| **GloVe**     | - Captura relaciones semánticas de manera más efectiva a través de una matriz de co-ocurrencia global. <br> - Buen rendimiento en tareas de similaridad semántica. | - Entrenamiento más complejo y lento comparado con Word2Vec. <br> - Requiere más memoria debido al uso de matrices grandes. |

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [13]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

import fasttext
import fasttext.util

fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')
vocabulario = list(midiccionario.keys())



# *********** Aquí termina la sección de agregar código *************

2000000

#**Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 9:**



Reporte del mejor modelo.


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**